In [1]:
course_metrics = {'course_name' : [],
                  'val_binary_accuracy' : [],
                  'test_f1_score' : []}

### Design the feed-forward neural net

In [2]:
import pandas as pd
import numpy as np

In [3]:
from keras.models import Sequential

from keras.layers import Dense, Activation, Dropout

from keras.losses import binary_crossentropy

from keras.optimizers import RMSprop

from keras.metrics import binary_accuracy

from keras.callbacks import EarlyStopping, ReduceLROnPlateau

from keras import backend as K

C:\Users\kimmcodxb\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [4]:
from sklearn.metrics import f1_score

In [23]:
def build(network_type=Sequential, 
          nb_initial_layer=64,
          dense_layer_lst=[64],
          nb_final_layer=1,
          dp_rate=0.2,
          learning_rate=1e-4):
    
    model = network_type()
    
    model.add(Dense(nb_initial_layer, input_shape=(features_train.shape[1], )))
    model.add(Activation('relu'))
    
    model.add(Dropout(dp_rate))
    
    for nb_units in dense_layer_lst:
        model.add(Dense(nb_units))
        model.add(Activation('relu'))
        model.add(Dropout(dp_rate))
        
    model.add(Dense(nb_final_layer))
    model.add(Activation('sigmoid'))
    
    model.compile(optimizer=RMSprop(lr=learning_rate),
                  loss=binary_crossentropy,
                  metrics=[binary_accuracy])
    
    return model

### CB22x - Ancient Greek Hero

In [6]:
cb22x_features_train = pd.read_feather('processed-final/HarvardXCB22x2013_Spring_features_train.feather')
cb22x_features_test = pd.read_feather('processed-final/HarvardXCB22x2013_Spring_features_test.feather')
cb22x_labels_train = pd.read_feather('processed-final/HarvardXCB22x2013_Spring_labels_train.feather')
cb22x_labels_test = pd.read_feather('processed-final/HarvardXCB22x2013_Spring_labels_test.feather')

In [7]:
features_train, features_test = cb22x_features_train.drop('index', axis=1), cb22x_features_test.drop('index', axis=1)
labels_train, labels_test = cb22x_labels_train.drop('index', axis=1), cb22x_labels_test.drop('index', axis=1)

In [8]:
features_train = np.array(features_train)
features_test = np.array(features_test)

labels_train = np.array(labels_train).ravel()
labels_test = np.array(labels_test).ravel()

In [9]:
features_train.shape

(24001, 18)

In [10]:
labels_train.shape

(24001,)

In [46]:
K.clear_session()

In [47]:
model = build(nb_initial_layer=32, dense_layer_lst=[32, 32, 32], dp_rate=0.3, learning_rate=1e-5)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 32)                640       
_________________________________________________________________
activation_1 (Activation)    (None, 32)                0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 32)                1056      
_________________________________________________________________
activation_2 (Activation)    (None, 32)                0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 32)                1056      
__________

In [48]:
model_output = model.fit(features_train, labels_train,
                         batch_size=128,
                         epochs=20,
                         validation_split=0.2,
                         callbacks=[EarlyStopping(patience=4), ReduceLROnPlateau(patience=4, min_lr=1e-6)])

Train on 108556 samples, validate on 27140 samples
Epoch 1/20
108556/108556 [==============================] - 3s 23us/step - loss: 2.2622 - binary_accuracy: 0.5344 - val_loss: 1.0604 - val_binary_accuracy: 0.6549
Epoch 2/20
108556/108556 [==============================] - 2s 19us/step - loss: 1.8580 - binary_accuracy: 0.5551 - val_loss: 0.8931 - val_binary_accuracy: 0.6584
Epoch 3/20
108556/108556 [==============================] - 2s 19us/step - loss: 1.5657 - binary_accuracy: 0.5586 - val_loss: 0.7930 - val_binary_accuracy: 0.6595
Epoch 4/20
108556/108556 [==============================] - 2s 19us/step - loss: 1.3612 - binary_accuracy: 0.5590 - val_loss: 0.7374 - val_binary_accuracy: 0.6593
Epoch 5/20
108556/108556 [==============================] - 2s 18us/step - loss: 1.2074 - binary_accuracy: 0.5568 - val_loss: 0.6987 - val_binary_accuracy: 0.6598
Epoch 6/20
108556/108556 [==============================] - 2s 19us/step - loss: 1.0755 - binary_accuracy: 0.5587 - val_loss: 0.6738 -

In [49]:
course_metrics['course_name'].append('HarvardCB22x_2013_Spring')
course_metrics['val_binary_accuracy'].append(model_output.history['val_binary_accuracy'][-1])
course_metrics['test_f1_score'].append(f1_score(model.predict_classes(features_test), labels_test))

### CS50x - Introduction to Compute Science I

In [41]:
cs50x_features_train = pd.read_feather('processed-final/HarvardXCS50x2012_features_train.feather')
cs50x_features_test = pd.read_feather('processed-final//HarvardXCS50x2012_features_test.feather')
cs50x_labels_train = pd.read_feather('processed-final//HarvardXCS50x2012_labels_train.feather')
cs50x_labels_test = pd.read_feather('processed-final//HarvardXCS50x2012_labels_test.feather')

In [42]:
features_train, features_test = cs50x_features_train.drop('index', axis=1), cs50x_features_test.drop('index', axis=1)
labels_train, labels_test = cs50x_labels_train.drop('index', axis=1), cs50x_labels_test.drop('index', axis=1)

features_train = np.array(features_train)
features_test = np.array(features_test)

labels_train = np.array(labels_train).ravel()
labels_test = np.array(labels_test).ravel()

In [43]:
features_train.shape

(135696, 19)

In [44]:
labels_train.shape

(135696,)

In [64]:
K.clear_session()

In [65]:
model = build(nb_initial_layer=32, dense_layer_lst=[32, 32, 32], dp_rate=0, learning_rate=1e-5)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 32)                640       
_________________________________________________________________
activation_1 (Activation)    (None, 32)                0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 32)                1056      
_________________________________________________________________
activation_2 (Activation)    (None, 32)                0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 32)                1056      
__________

In [66]:
model_output = model.fit(features_train, labels_train,
                         batch_size=128,
                         epochs=100,
                         validation_split=0.2,
                         callbacks=[EarlyStopping(patience=4), ReduceLROnPlateau(patience=4, min_lr=1e-6)])

Train on 108556 samples, validate on 27140 samples
Epoch 1/100
108556/108556 [==============================] - 2s 15us/step - loss: 1.0187 - binary_accuracy: 0.4570 - val_loss: 0.6634 - val_binary_accuracy: 0.6301
Epoch 2/100
108556/108556 [==============================] - 1s 12us/step - loss: 0.6369 - binary_accuracy: 0.6521 - val_loss: 0.6265 - val_binary_accuracy: 0.6567
Epoch 3/100
108556/108556 [==============================] - 1s 12us/step - loss: 0.6252 - binary_accuracy: 0.6603 - val_loss: 0.6232 - val_binary_accuracy: 0.6592
Epoch 4/100
108556/108556 [==============================] - 1s 13us/step - loss: 0.6234 - binary_accuracy: 0.6623 - val_loss: 0.6211 - val_binary_accuracy: 0.6597
Epoch 5/100
108556/108556 [==============================] - 1s 12us/step - loss: 0.6224 - binary_accuracy: 0.6625 - val_loss: 0.6216 - val_binary_accuracy: 0.6606
Epoch 6/100
108556/108556 [==============================] - 1s 13us/step - loss: 0.6216 - binary_accuracy: 0.6628 - val_loss: 0.

108556/108556 [==============================] - 1s 12us/step - loss: 0.6038 - binary_accuracy: 0.6795 - val_loss: 0.6023 - val_binary_accuracy: 0.6820
Epoch 51/100
108556/108556 [==============================] - 1s 12us/step - loss: 0.6036 - binary_accuracy: 0.6801 - val_loss: 0.6027 - val_binary_accuracy: 0.6809
Epoch 52/100
108556/108556 [==============================] - 1s 11us/step - loss: 0.6035 - binary_accuracy: 0.6797 - val_loss: 0.6022 - val_binary_accuracy: 0.6818
Epoch 53/100
108556/108556 [==============================] - 1s 13us/step - loss: 0.6033 - binary_accuracy: 0.6804 - val_loss: 0.6021 - val_binary_accuracy: 0.6822
Epoch 54/100
108556/108556 [==============================] - 1s 12us/step - loss: 0.6032 - binary_accuracy: 0.6799 - val_loss: 0.6020 - val_binary_accuracy: 0.6815
Epoch 55/100
108556/108556 [==============================] - 1s 11us/step - loss: 0.6030 - binary_accuracy: 0.6804 - val_loss: 0.6025 - val_binary_accuracy: 0.6810
Epoch 56/100
108556/108

In [67]:
course_metrics['course_name'].append('HarvardCB50x_2012')
course_metrics['val_binary_accuracy'].append(model_output.history['val_binary_accuracy'][-1])
course_metrics['test_f1_score'].append(f1_score(model.predict_classes(features_test), labels_test))

### ER22x - Justice

In [68]:
er22x_features_train = pd.read_feather('processed-final/HarvardXER22x2013_Spring_features_train.feather')
er22x_features_test = pd.read_feather('processed-final/HarvardXER22x2013_Spring_features_test.feather')
er22x_labels_train = pd.read_feather('processed-final/HarvardXER22x2013_Spring_labels_train.feather')
er22x_labels_test = pd.read_feather('processed-final/HarvardXER22x2013_Spring_labels_test.feather')

In [ ]:
features_train, features_test = er22x_features_train.drop('index', axis=1), er22x_features_test.drop('index', axis=1)
labels_train, labels_test = er22x_labels_train.drop('index', axis=1), er22x_labels_test.drop('index', axis=1)

In [ ]:
features_train = np.array(features_train)
features_test = np.array(features_test)

labels_train = np.array(labels_train).ravel()
labels_test = np.array(labels_test).ravel()